In [1]:
#!pip install geoplot
#!pip install contextily
#!pip install altair

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import geopandas as gpd 
import nltk
nltk.download('punkt')
#import contextily as ctx
#import geoplot as gplt
#import geoplot.crs as gcrs
#import folium
#import seaborn as sns
#import altair as alt 

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Part I : Obtention du dataframe 
## 1.1.Récupération des loyers, des surfaces et de la localisation des résidences

In [3]:
df = pd.read_csv("fr_crous_logement_france_entiere.csv", sep =';')


In [4]:
df = df[["title", "infos", "address" ,"geocalisation", "regions"]]

In [ ]:
from functions_for_modelling import *

df = get_loyer(df)
df = get_surface(df)
df = get_localisation(df) 
df

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 1.2.Récupération des loyers privés des communes (avec la géolocalisation des communes)

In [ ]:
df_private = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/8fac6fb7-cd07-4747-8e0b-b101c476f0da", encoding= 'unicode_escape', sep = ";")


On ne garde que le code INSEE, le nom de la ville et le loyer supérieur par m^2


In [ ]:
df_private = df_private[["INSEE", "LIBGEO","upr.IPm2"]]

On renomme la colonne INSEE en codgeo pour pouvoir la merger avec la carte des contours des villes.

In [ ]:
df_private.rename(columns = {'INSEE':'codgeo'}, inplace = True)

In [ ]:
"""
On rajoute un 0 devant le code INSEE quand il ne fait que quatre chiffres (pour uniformiser avec l'autre base).
"""
for i in range(0,df_private.shape[0]):
    if  len(df_private["codgeo"][i]) == 4: 
            df_private["codgeo"][i] = "0" +  df_private["codgeo"][i]

In [ ]:
df_private = df_private.set_index('codgeo') #on réindex le dataframe avec le code INSEE pour merger après. 

On récuppère les contours géographique des communes.(prends du temps)

In [ ]:
gdf_communes_boundaries = gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/e9391593-fa95-4153-aabe-87ca84d197e9")

On ne garde que le code INSEE et les données polygoniales. 

In [ ]:
gdf_communes_boundaries.rename(columns = {'INSEE_COM':'codgeo'}, inplace = True)
gdf_communes_boundaries = gdf_communes_boundaries[["codgeo", "geometry"]]
gdf_communes_boundaries = gdf_communes_boundaries.set_index('codgeo')


On concatène les deux datafarmes

In [ ]:
gdf_private = pd.concat([df_private,gdf_communes_boundaries], axis = 1, join = "inner")

## 1.3. Jointures spatiales

In [ ]:

#gdf_private['geometry'] = gdf_private['geometry'].apply(wkt.loads) #on transforme en format wkt (sinon ça ne marche pas)
gdf_private = gpd.GeoDataFrame(gdf_private)  #on transforme en geodataframe 
df = gpd.sjoin(df, gdf_private, how='left', op='within') #on réalise la jointure à gauche avec within 
#i.e si le point est dans le polynôme. 

In [ ]:
del df['index_right'] #on supprime l'index right pour la prochaine jointure
df

## 1.4 Ajout des départements 

In [ ]:
gdf_dep = gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/eb36371a-761d-44a8-93ec-3d728bec17ce")
gdf_dep.rename(columns = {'nom':'departement'}, inplace = True)
gdf_dep = gdf_dep[["departement", "geometry"]]
gdf_dep

In [ ]:
df = gpd.sjoin(df, gdf_dep, how='left', op='within')


In [ ]:
df

In [ ]:
del df['index_right'] #on supprime l'index right pour la prochaine jointure
df

## 1.5. Association de résidences à une école 

In [ ]:
df.crs = "epsg:4326" #on définit la projection de notre dataframe

In [ ]:
"""
Calcul d'un cercle de 20 km autour de la résidence
"""
df1 = df
df1 = df1.to_crs(epsg = 3395) #On prend une projection en mètres
df1['geometry'] = df1['geometry'].buffer(20000) #On prend un rayon de 20km 
df1 = df1.to_crs(4326) #on repasse dans la projection uselle 
df1

In [ ]:
"""
Import de notre base de données des écoles 
"""
df_schools = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/0c713161-26fb-415e-ac1d-8769125f338d", sep = ";")

In [ ]:
df_schools = df_schools[df_schools["annee_universitaire"] == "2021-22" ] #on ne prend que l'année 2021-22
df_schools = df_schools[["etablissement_lib", "effectif_sans_cpge", "etablissement_code_commune", "etablissement_commune"]]

In [ ]:
df_schools

In [ ]:
df_schools.rename(columns = {'etablissement_code_commune':'codgeo'}, inplace = True) #On renomme la colonne pour la jointure


In [ ]:
index_with_nan = df_schools.index[df_schools.isnull().any(axis=1)]
df_schools.drop(index_with_nan,0, inplace=True) #on enlève les valeurs manquantes

In [ ]:
df_schools.set_index('codgeo', inplace = True)
df_schools

In [ ]:
df_schools = df_schools.join(gdf_communes_boundaries)
df_schools

In [ ]:
df_schools = gpd.GeoDataFrame(df_schools)
df_schools.crs = "epsg:4326"

In [ ]:
"""
essai sur la résidence de Cergy 
"""
a = df_schools.within(df1["geometry"][1])
i = 0 
for element in a : 
    if element == True: 
        print(df_schools['etablissement_lib'][i])
    i = i + 1

In [ ]:
Schools = [] #on crée une liste pour stocker la liste des écoles 
Nbstudents = [] #on crée une liste pour stocker la liste du nombre d'étudiants
Nbstudents_total  = []
for residence in df1["geometry"]: #on prend pour chaque résidence le cercle de 5 km autour de cette résidence
    S = [] #stocke les écoles pour la résidence
    N = [] #idem pour le nb d'étudiants 
    i = 0 #index 
    a = df_schools.within(residence) #On utilise gpd.within qui regarde pour chaque élèment si sa coordonnée est dans le polygone résidence
    for element in a : #on regarde pour chaque élèment de a si l'école est dans le cercle de 5km autour de la résidence
        if element == True: 
            S.append(df_schools['etablissement_lib'][i]) #on ajoute l'école de l'index i 
            N.append(df_schools['effectif_sans_cpge'][i]) #idem pour le nombre d'étudiants
        i = i + 1
    Schools.append(S) 
    Nbstudents.append(N)
    Nbstudents_total.append(sum(N))
df['Schools'] = Schools
df['Nbstudents'] = Nbstudents
df['Nbstudents_total'] = Nbstudents_total
df
       

# II. Visualisation 

## 2.1 Portrait économique et statistique des résidences

In [ ]:
df['prix_par_m2_min'] = df['Min Loyer']/df['Min Surface'] # On crée une nouvelle variable prix_par_m2_min 

In [ ]:
df["upr.IPm2"] = df["upr.IPm2"].apply(lambda x : x.replace(',', '.')) #on remplace les , par des points pour ensuite transformet cette colonne en float


In [ ]:
df["upr.IPm2"] = df["upr.IPm2"].astype(float)

In [ ]:
df.groupby(['departement']).mean().sort_values('prix_par_m2_min')

## Ajout de la colonne 'prix_par_m2_max'(qui correspond le plus souvent à des logements en colocation (T2/T3/T4))

In [ ]:
df['prix_par_m2_max'] = df['Max Loyer']/df['Max Surface'] # On crée une nouvelle variable prix_par_m2_min 

In [ ]:
df.groupby(['departement']).mean().sort_values('prix_par_m2_max')

In [ ]:
df_max = df.groupby(['departement']).mean().sort_values('prix_par_m2_max')
df_max = df_max.loc[(df_max['prix_par_m2_min']>1)] 

In [ ]:
df_max['departement'] = df_max.index #je crée une colonne qui correspond à l'index (la colonne regions)

## Ajout de la colonne 'prix_par_m2_mean'(qui correspond à la moyenne des loyers des logements porposés) 

In [ ]:
df['prix_par_m2_mean'] = df['Mean Loyer']/df['Mean Surface'] # On crée une nouvelle variable prix_par_m2_mean

# Modélisation 

On commence par enlever les NaN du dataframe qui bloquent la regression linéaire

In [ ]:
from sklearn.linear_model import LinearRegression
import math
from sklearn.model_selection import train_test_split

In [ ]:
#on commence par retirer les NaN pour que les régressions fonctionnent
df.isnull().sum()
df2 = df
index_with_nan = df2.index[df2.isnull().any(axis=1)]
print(index_with_nan.shape)
df2.drop(index_with_nan,0, inplace=True)
print(df2[['Mean Loyer', 'Mean Surface']].isnull().sum())
df2 = df2.reset_index()
df2

### Regressions linéaires OLS

#### https://www.statology.org/simple-linear-regression-in-python/

In [ ]:
print(df2['Mean Surface'].describe()) #on vérifie qu'il n'y a pas de valeurs incohérentes (min, max) qui viendraient fausser les résultats
print(df2['Mean Loyer'].describe())

In [ ]:
plt.scatter(df2['Mean Surface'], df2['Mean Loyer']) #on regarde la répartition des loyers en fonction de la surface.
plt.title('Loyer en fonction de la surface en mètre carrés')
plt.xlabel('Surface (en mètre carré)')
plt.ylabel('Loyer (en euros)')
plt.show()


In [ ]:
#define response variable
y = df2['Mean Loyer']

#define explanatory variable
x = df2['Mean Surface']

#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model = sm.OLS(y, x).fit()

#view model summary
print(model.summary())

In [ ]:
#le R² est faible: la variable explicative (ici la surface moyenne) n'est pas très utile pour prédire le résultat. 

In [ ]:
#define figure size
fig = plt.figure(figsize=(12,8))

#produce residual plots
fig = sm.graphics.plot_regress_exog(model, 'Mean Surface', fig=fig)

In [ ]:
#define residuals
res = model.resid

#create Q-Q plot
fig = sm.qqplot(res, fit=True, line="45")
plt.show() 


### Régression linéaire du prix par m² des résidences Crous ('Mean Loyer'/'Mean Surface') expliqué par le prix au m² des résidences privées

In [ ]:
plt.scatter(df2['upr.IPm2'], df2['Mean Loyer']/df2['Mean Surface'])
plt.title('prix par m² en résidence privée vs. prix par m² en résidence crous')
plt.xlabel('résidences privées')
plt.ylabel('résidences crous')
plt.show()

In [ ]:
from functions_for_modelling import *
get_OLS_reg(df2['upr.IPm2'],df2['Mean Loyer']/df2['Mean Surface'])

In [ ]:
Y = df2['Mean Loyer']/df2['Mean Surface'] #on définit la variable Y, le prix du m² moyen des résidences crous
X = df2[['upr.IPm2']] #la variable explicative est le prix au m² des résidences privées

X = sm.add_constant(X) #on ajoute une constante au vecteur des variables explicatives

#fit linear regression model
model = sm.OLS(Y, X).fit()

#view model summary
print(model.summary())

In [ ]:
#le R² est très proche de zéro: comme on pouvait s'y attendre, le prix par mètre carré des loyers privés ne permet pas d'expliquer le prix par mètre carré des loyers des résidences du crous

In [ ]:
#regardons si les résidus semblent homoscedastiques ou pas

In [ ]:
#define figure size
fig = plt.figure(figsize=(12,8))

#produce residual plots
fig = sm.graphics.plot_regress_exog(model, 'upr.IPm2', fig=fig)

In [ ]:
#define residuals
res = model.resid

#create Q-Q plot
fig = sm.qqplot(res, fit=True, line="45")
plt.show() 

### Régression linéaire du loyer en fonction du nombre d'étudiants total

In [ ]:
#on crée un nouveau dataframe df_students où on ne garde que les résidences avec plus de 1000 étudiants
i = df2[df2['Nbstudents_total']<=1000].index.values
df_students = df2.drop(i,axis=0) 

In [ ]:
plt.scatter(df_students['Nbstudents_total'], df_students['Mean Loyer']) #on regarde la répartition des loyers en fonction de la surface.
plt.title("Loyer en fonction du nombre d'étudiants présents")
plt.xlabel("Nombre d'étudiants")
plt.ylabel('Loyer (en euros)')
plt.show()

In [ ]:
get_OLS_reg(df_students['Nbstudents_total'],df_students['Mean Loyer'])

### Régression longue du prix par mètre carré des résidences Crous ('Mean Loyer'/'Mean Surface') expliqué par le nombre d'étudians total et le prix au mètre carré des résidences privées

In [ ]:
get_OLS_reg(df_students[['Nbstudents_total','upr.IPm2']],df_students['Mean Loyer']/df_students['Mean Surface'])

In [ ]:
Y = df_students['Mean Loyer']/df_students['Mean Surface'] #on définit la variable Y, le prix du m² moyen des résidences crous
X = df_students[['Nbstudents_total','upr.IPm2']] #la variable explicative est le prix au m² des résidences privées

X = sm.add_constant(X) #on ajoute une constante au vecteur des variables explicatives

#fit linear regression model
model = sm.OLS(Y, X).fit()

#view model summary
print(model.summary())

In [ ]:
#define residuals
res = model.resid

#create Q-Q plot
fig = sm.qqplot(res, fit=True, line="45")
plt.show() 

In [ ]:
#on obtient presque le même graphique que pour la régression simple des prix au mètre carré des résidences crous sur les prix au mètre carré des résidences privées.
#le coefficient associé au nombre d'étudiants total est extrêmement faible.

## Regressions linéaires avec le module LinearRegression de sklearn

### On reprend la première régression effectuée plus haut: le loyer moyen expliqué par la surface moyenne

In [ ]:
get_sklearn_regression(df2['Mean Surface'],df2['Mean Loyer'],'Loyer_moyen','Surface_moyenne')

In [ ]:
var_explicatives = df2['Mean Surface']
YLoyer = df2['Mean Loyer']
#on transforme Y et X en matrices colonnes
YLoyer = np.array(YLoyer).reshape(-1,1)
var_explicatives = (np.array(var_explicatives).reshape(-1,1))

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(var_explicatives,YLoyer,test_size=0.3,train_size=0.7)
lin= LinearRegression()
reg = lin.fit(X_train,Y_train)
pred_train = lin.predict(X_train)
pred_test = lin.predict(X_test)

coefficients_sans_cst= reg.coef_
r_2 = reg.score(X_train,Y_train)
print(r_2)
print(coefficients_sans_cst)

In [ ]:
#On trouve un résultat légèrement différent de celui avec sm.OLS

In [ ]:

plt.scatter(X_train, Y_train, color='red') # plotting the observation line
 
plt.plot(X_train, lin.predict(X_train), color='blue') # plotting the regression line
 
plt.title("Loyer moyen vs Surface moyenne (Training set)") # stating the title of the graph
 
plt.xlabel("Surface moyenne") # adding the name of x-axis
plt.ylabel("Loyer moyen") # adding the name of y-axis
plt.show() # specifies end of graph


In [ ]:
#Deuxième essai: regression linéaire du min Loyer sur min surface 

In [ ]:
df2[['Min Loyer', 'Min Surface']].isnull().sum()

In [ ]:
X_min_surface = df2['Min Surface'] #la variable explicative
Y_min_Loyer = df2['Min Loyer']
#on transforme Y et X en array que l'on reformate
Y_min_Loyer = np.array(Y_min_Loyer).reshape(-1,1)
X_min_surface = (np.array(X_min_surface).reshape(-1,1))

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_min_surface,Y_min_Loyer,test_size=0.2,train_size=0.8)
lin= LinearRegression()
reg = lin.fit(X_train,Y_train)
pred_train = lin.predict(X_train)
pred_test = lin.predict(X_test)

coefficients_sans_cst= reg.coef_
r_2 = reg.score(X_train,Y_train)
print(r_2)
print(coefficients_sans_cst) 

In [ ]:
#avec la régression du min Loyer sur min Surface, on a un R² peu significatif

#### Regression linéaire du Loyer sur le nombre total d'étudiants

In [ ]:
X_Nbstudents = df_students['Nbstudents_total'] #la variable explicative
YLoyer = df_students['Mean Loyer']
#on transforme Y et X en array que l'on reformate
YLoyer = np.array(YLoyer).reshape(-1,1)
X_Nbstudents = (np.array(X_Nbstudents).reshape(-1,1))

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_Nbstudents,YLoyer,test_size=0.2,train_size=0.8)
lin= LinearRegression()
reg = lin.fit(X_train,Y_train)
pred_train = lin.predict(X_train)
pred_test = lin.predict(X_test)

coefficients_sans_cst= reg.coef_
r_2 = reg.score(X_train,Y_train)
print(r_2)
print(coefficients_sans_cst) 

In [ ]:
plt.scatter(X_train, Y_train, color='red') # plotting the observation line
 
plt.plot(X_train, lin.predict(X_train), color='blue') # plotting the regression line
 
plt.title("Loyer moyen vs Nombre d'étudiants total (Training set)") # stating the title of the graph
 
plt.xlabel("Nb d'étudiants total") # adding the name of x-axis
plt.ylabel("Loyer moyen") # adding the name of y-axis
plt.show() # specifies end of graph

In [ ]:
#Regression linéaire du Loyer sur le LOG du nombre total d'étudiants

In [ ]:
X_log_Nbstudents = np.log(df_students['Nbstudents_total']) #la variable explicative
YLoyer = df_students['Mean Loyer']
#on transforme Y et X en array que l'on reformate
YLoyer = np.array(YLoyer).reshape(-1,1)
X_log_Nbstudents = (np.array(X_log_Nbstudents).reshape(-1,1))

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_log_Nbstudents,YLoyer,test_size=0.2,train_size=0.8)
lin= LinearRegression()
reg = lin.fit(X_train,Y_train)
pred_train = lin.predict(X_train)
pred_test = lin.predict(X_test)

coefficients_sans_cst= reg.coef_
r_2 = reg.score(X_train,Y_train)
print(r_2)
print(coefficients_sans_cst) 

In [ ]:
plt.scatter(X_train, Y_train, color='red') # plotting the observation line
 
plt.plot(X_train, lin.predict(X_train), color='blue') # plotting the regression line
 
plt.title("Loyer moyen en fonction du logarithme du nombre d'étudiants total (Training set)") # stating the title of the graph
 
plt.xlabel("Logarithme du nombre d'étudiants total") # adding the name of x-axis
plt.ylabel("Loyer moyen") # adding the name of y-axis
plt.show() # specifies end of graph

In [ ]:
#régression linéaire du 